In [148]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

chrome_options = Options()
chrome_options.add_argument('start-maximized')  #--headless


driver = webdriver.Chrome('./chromedriver.exe',options=chrome_options)
driver.get('https://mail.ru/')

#вход в почту
elem = driver.find_element_by_id('mailbox:login')
elem.send_keys('study.ai_172@mail.ru')



#elem = driver.find_element_by_id('user_password')
#elem.send_keys('Password172')

#elem.send_keys(Keys.ENTER)

#time.sleep(0.9)
#profile = driver.find_element_by_class_name('avatar')
#profile_link = profile.get_attribute('href')
#driver.get(profile_link)

#edit_profile = driver.find_element_by_class_name('text-sm')
#edit_profile_link = edit_profile.get_attribute('href')
#driver.get(edit_profile_link)

select = driver.find_element_by_id('mailbox:domain')
options = select.find_elements_by_tag_name('option')

for option in options:
     if option.text == '@mail.ru':
        option.click()

button = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.ID,'mailbox:submit'))
        )

button.click()

time.sleep(3)
elem = WebDriverWait(driver,60).until(
            EC.presence_of_element_located((By.ID,'mailbox:password'))
        )

elem.send_keys('NextPassword172')

button = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.ID,'mailbox:submit'))
        )

button.click()

button = WebDriverWait(driver,60).until(
            EC.presence_of_element_located((By.CLASS_NAME,'js-letter-list-item'))
        )

#получаем письма из почты
#last_article = ''
#page = 0
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#while True:
#    time.sleep(5)
#    articles = driver.find_elements_by_class_name('js-letter-list-item')
#    print(len(articles))
#    if articles[-1] != last_article:
#        page+=1
#        print(articles[-1].text)
#        last_article = articles[-1]
#        actions = ActionChains(driver)
#        actions.move_to_element(articles[-1])   #.key_down(Keys.CTRL).key_down(Keys.ENTER).key_up(Keys.Enter)
#        actions.perform()
#        time.sleep(3) 
#    else:
#        print(articles[-1].text)
#        break




time.sleep(3)
article = driver.find_element_by_class_name('js-letter-list-item')
article.click()


letters = []
while True:
    time.sleep(1)
    try:
        #button = driver.find_element_by_xpath('//span[@title="Следующее"]')  
        
        button = WebDriverWait(driver, 10).until(
             EC.presence_of_element_located(( By.XPATH, '//span[@title="Следующее"]' ))
        )
        class_name = button.get_attribute('class')
        if not ('disabled' in class_name):
  
            actions = ActionChains(driver)
            actions.key_down(Keys.CONTROL).key_down(Keys.ARROW_DOWN).key_up(Keys.ARROW_DOWN)
            actions.perform()
            

            elem = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'letter-contact'))
            )
            sender = elem.text 

            elem = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'letter__date'))
            )
            date= elem.text
       
            elem = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'thread__subject'))
            )
            subject = elem.text

            elem = WebDriverWait(driver, 3).until(
                 EC.presence_of_element_located((By.CLASS_NAME, 'letter__body'))
            )
            full_text = elem.text
            letter = {}
            letter['full_text'] = full_text
            letter['subject'] = subject
            letter['date'] = date
            letter['sender'] = sender
            letters.append(letter)
        else:
    
            break
    except:
        break
    

    
#вставляем данные в БД
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['mails']

mails = db.mails
mails.insert_many(letters)















In [150]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import json

chrome_options = Options()
chrome_options.add_argument('start-maximized')  #--headless


driver = webdriver.Chrome('./chromedriver.exe',options=chrome_options)
driver.get('https://www.mvideo.ru/')


time.sleep(7)

#class_names=['gallery-layout', 'sel-hits-block']



#ищем нужную кнопку        
button_list = driver.find_elements_by_class_name('sel-hits-button-next')
button = button_list[2] 


        
while True:
    button.click()
    time.sleep(1)
    class_name = button.get_attribute('class')
    if 'disabled' in class_name:
        break 

#ищем блок Хиты продаж

goods = driver.find_elements_by_class_name('sel-hits-block')
good = goods[1]

popular_list = []      
list_elems = good.find_elements_by_class_name("gallery-list-item")
price = 0
for one_elem in list_elems:
    popular_item = {}  
    one_elem_a = one_elem.find_element_by_tag_name('a')  
    product_info = one_elem_a.get_attribute('data-product-info')
    product_info = json.loads(product_info)
    #print(product_info)
    name = product_info['productName']
    price = product_info['productPriceLocal']
    popular_item['name'] = name 
    popular_item['price'] = float(price) 
    popular_list.append(popular_item)
 
        
        
print(popular_list)

               


[{'name': 'Смартфон Samsung Galaxy S10 Оникс', 'price': 49990.0}, {'name': 'Смартфон Apple iPhone XR 64GB Black (MRY42RU/A)', 'price': 47990.0}, {'name': 'Смартфон Samsung Galaxy S20+ Black (SM-G985F/DS)', 'price': 74990.0}, {'name': 'Смартфон Redmi Note 8T 64GB Starscape Blue', 'price': 13990.0}, {'name': 'Смартфон Samsung Galaxy Note10 Aura Glow (SM-N970F)', 'price': 64990.0}, {'name': 'Смартфон Samsung Galaxy Note10 Black (SM-N970F)', 'price': 64990.0}, {'name': 'Смартфон Samsung Galaxy Note10 Red (SM-N970F)', 'price': 64990.0}, {'name': 'Смартфон Samsung Galaxy Note10+ Aura Glow (SM-N975F)', 'price': 74990.0}, {'name': 'Смартфон Samsung Galaxy Note10+ Black (SM-N975F)', 'price': 74990.0}, {'name': 'Смартфон Samsung Galaxy Note10+ White (SM-N975F)', 'price': 74990.0}, {'name': 'Смартфон Apple iPhone 11 128GB Black (MWM02RU/A)', 'price': 64990.0}, {'name': 'Наушники Apple AirPods w/Charging Case (MV7N2RU/A)', 'price': 10990.0}, {'name': 'Смартфон Samsung Galaxy A51 64GB Black (SM-A51

In [ ]:
#вставляем данные в БД
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['vacancy_db']

mvideo = db.mvideo
mvideo.insert_many(popular_list)